In [ ]:
get_ipython().system('pip install "folium>=0.12" matplotlib mapclassify')

In [ ]:
from pathlib import Path
import scipy as sp
from google.colab import drive
import pandas as pd
import geopandas
import matplotlib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import requests

In [ ]:
drive.mount("/content/drive", force_remount=True)
dataset_path = list(Path("/content/drive/MyDrive/Mestrado/Disciplinas/2_semestre/Ciência_de_Dados/Trabalho/Dataset/Ideia_simplificada/dataset_final/").glob("*.csv"))

dataframe_leite = pd.read_csv(dataset_path[0])

Mounted at /content/drive


## Localização Ebrapa e Rede brasileira de qualidade do leite

In [ ]:
coordenadas_cidades = {
    "Rio Branco": [-9.9754, -67.8249],
    "Campinas": [-22.9099, -47.0626],
    "Seropédica": [-22.7444, -43.7090],
    "Brasília": [-15.7975, -47.8919],
    "Fortaleza": [-3.7172, -38.5434],
    "Rio de Janeiro": [-22.9068, -43.1729],
    "Dourados": [-22.2232, -54.8064],
    "Sinop": [-11.8616, -55.5056],
    "Campina Grande": [-7.2195, -35.8828],
    "Maceió": [-9.6662, -35.7351],
    "Macapá": [0.0355, -51.0705],
    "Manaus": [-3.1190, -60.0217],
    "Belém": [-1.4558, -48.4902],
    "Santo Antônio de Goiás": [-16.4827, -49.3127],
    "Sobral": [-3.6846, -40.3489],
    "Pelotas Sistema de Pesquisa e Desenvolvimento em Pecuária Leiteira (Sispel)": [-31.7654, -52.3376],
    "Colombo": [-25.2917, -49.2242],
    "Campo Grande": [-20.4697, -54.6201],
    "Sede da Embrapa Gado de Leite(Juiz de Fora)": [-21.7661, -43.3502],
    "São Carlos": [-22.0050, -47.8888],
    "São Luís": [-2.5307, -44.3068],
    "Cruz das Almas": [-12.6769, -39.1025],
    "Jaguariúna": [-22.7041, -46.9870],
    "Teresina": [-5.0919, -42.8034],
    "Sete Lagoas": [-19.4660, -44.2466],
    "Corumbá": [-19.0102, -57.6538],
    "Bagé": [-31.3285, -54.1070],
    "Palmas": [-10.2491, -48.3243],
    "Porto Velho": [-8.7612, -63.9039],
    "Boa Vista": [2.8235, -60.6758],
    "Petrolina": [-9.3960, -40.5065],
    "Londrina": [-23.3045, -51.1696],
    "Concórdia": [-27.2350, -52.0257],
    "Aracaju": [-10.9472, -37.0731],
    "Passo Fundo": [-28.2628, -52.4088],
    "Bento Gonçalves": [-29.1691, -51.5178]
}

lista_cidades = [
    "Rio Branco", "Campinas", "Seropédica", "Brasília", "Fortaleza",
    "Rio de Janeiro", "Dourados", "Sinop", "Campina Grande", "Maceió",
    "Macapá", "Manaus", "Belém", "Santo Antônio de Goiás", "Brasília",
    "Sobral", "Brasília", "Pelotas Sistema de Pesquisa e Desenvolvimento em Pecuária Leiteira (Sispel)", "Colombo", "Campo Grande",
    "Sede da Embrapa Gado de Leite(Juiz de Fora)", "Brasília", "São Carlos", "São Luís", "Cruz das Almas",
    "Jaguariúna", "Teresina", "Sete Lagoas", "Corumbá", "Bagé",
    "São Carlos", "Palmas", "Brasília", "Porto Velho", "Boa Vista",
    "Petrolina", "Londrina", "Rio de Janeiro", "Concórdia", "Aracaju",
    "Campinas", "Passo Fundo", "Bento Gonçalves"
]

locais = {
    "Cariacica/ES": [-20.2638, -40.4206],
    "Goiânia/GO": [-16.6869, -49.2648],
    "Belo Horizonte/MG": [-19.9167, -43.9345],
    "Curitiba/PR": [-25.4284, -49.2733],
    "Passo Fundo/RS": [-28.2628, -52.4067],
    "Lajeado/RS": [-29.4669, -51.9614],
    "Pelotas/RS": [-31.7719, -52.3426],
    "Frederico Westphalen/RS": [-27.3592, -53.3969],
    "Piracicaba/SP": [-22.7345, -47.6481]
}

In [ ]:
import json
import folium
from folium import plugins
import pandas as pd

import folium
import requests

geojson_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [-53.8825, -15.7939]
            },
            "properties": {
                "region": "Centro-Oeste",
                "country": "Brazil",
                "timeseries": [
                    {"year": 2000, "production_millions": 3.08, "avg_temp_c": 24.37, "avg_precip_mm": 82.55, "avg_producer_price": 2.468, "net_price": 0.5418},
                    {"year": 2001, "production_millions": 3.246, "avg_temp_c": 23.83, "avg_precip_mm": 108.0, "avg_producer_price": 2.362, "net_price": 0.5433},
                    {"year": 2002, "production_millions": 3.46, "avg_temp_c": 24.47, "avg_precip_mm": 89.0, "avg_producer_price": 2.326, "net_price": 0.5851},
                    {"year": 2003, "production_millions": 3.535, "avg_temp_c": 24.40, "avg_precip_mm": 86.65, "avg_producer_price": 2.326, "net_price": 0.6486},
                    {"year": 2004, "production_millions": 3.62, "avg_temp_c": 24.50, "avg_precip_mm": 76.15, "avg_producer_price": 2.362, "net_price": 0.4862},
                    {"year": 2005, "production_millions": 3.778, "avg_temp_c": 24.51, "avg_precip_mm": 65.8, "avg_producer_price": 2.362, "net_price": 0.4821},
                    {"year": 2006, "production_millions": 3.722, "avg_temp_c": 25.09, "avg_precip_mm": 130.2, "avg_producer_price": 2.362, "net_price": 0.4465},
                    {"year": 2007, "production_millions": 3.808, "avg_temp_c": 24.64, "avg_precip_mm": 76.15, "avg_producer_price": 2.66, "net_price": 0.6389},
                    {"year": 2008, "production_millions": 4.055, "avg_temp_c": 24.52, "avg_precip_mm": 76.75, "avg_producer_price": 2.48, "net_price": 0.6552},
                    {"year": 2009, "production_millions": 4.222, "avg_temp_c": 24.45, "avg_precip_mm": 86.5, "avg_producer_price": 2.3, "net_price": 0.6219},
                    {"year": 2010, "production_millions": 4.45, "avg_temp_c": 24.44, "avg_precip_mm": 82.7, "avg_producer_price": 2.46, "net_price": 0.68},
                    {"year": 2011, "production_millions": 4.777, "avg_temp_c": 24.43, "avg_precip_mm": 83.0, "avg_producer_price": 2.44, "net_price": 0.7999},
                    {"year": 2012, "production_millions": 4.818, "avg_temp_c": 24.47, "avg_precip_mm": 84.4, "avg_producer_price": 2.13, "net_price": 0.8294},
                    {"year": 2013, "production_millions": 5.016, "avg_temp_c": 24.29, "avg_precip_mm": 85.2, "avg_producer_price": 2.33, "net_price": 0.9916},
                    {"year": 2014, "production_millions": 4.944, "avg_temp_c": 24.29, "avg_precip_mm": 87.1, "avg_producer_price": 2.15, "net_price": 0.9722},
                    {"year": 2015, "production_millions": 4.604, "avg_temp_c": 24.39, "avg_precip_mm": 85.6, "avg_producer_price": 1.95, "net_price": 0.9690},
                    {"year": 2016, "production_millions": 3.972, "avg_temp_c": 24.42, "avg_precip_mm": 80.8, "avg_producer_price": 2.22, "net_price": 1.2713},
                    {"year": 2017, "production_millions": 3.966, "avg_temp_c": 24.41, "avg_precip_mm": 79.6, "avg_producer_price": 1.96, "net_price": 1.0730},
                    {"year": 2018, "production_millions": 4.092, "avg_temp_c": 24.40, "avg_precip_mm": 81.25, "avg_producer_price": 2.19, "net_price": 1.3197},
                    {"year": 2019, "production_millions": 4.14, "avg_temp_c": 24.48, "avg_precip_mm": 80.0, "avg_producer_price": 2.27, "net_price": 1.4138},
                    {"year": 2020, "production_millions": 4.113, "avg_temp_c": 24.48, "avg_precip_mm": 79.8, "avg_producer_price": 2.64, "net_price": 1.8288},
                    {"year": 2021, "production_millions": 3.982, "avg_temp_c": 24.48, "avg_precip_mm": 79.8, "avg_producer_price": 2.38, "net_price": 2.1813},
                    {"year": 2022, "production_millions": 3.81, "avg_temp_c": 24.45, "avg_precip_mm": 78.8, "avg_producer_price": 2.73, "net_price": 2.8122},
                    {"year": 2023, "production_millions": 3.774, "avg_temp_c": 24.50, "avg_precip_mm": 79.4, "avg_producer_price": 2.51, "net_price": 2.4944}
                ]
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [-43.9378, -19.9167]
            },
            "properties": {
                "region": "Sudeste",
                "country": "Brazil",
                "timeseries": [
                    {"year": 2000, "production_millions": 8.574, "avg_temp_c": 18.93, "avg_precip_mm": 74.24, "avg_producer_price": 2.505, "net_price": 0.5458},
                    {"year": 2001, "production_millions": 8.573, "avg_temp_c": 18.88, "avg_precip_mm": 133.5, "avg_producer_price": 2.399, "net_price": 0.5540},
                    {"year": 2002, "production_millions": 8.746, "avg_temp_c": 18.64, "avg_precip_mm": 77.9, "avg_producer_price": 2.37, "net_price": 0.5900},
                    {"year": 2003, "production_millions": 8.934, "avg_temp_c": 20.04, "avg_precip_mm": 48.05, "avg_producer_price": 2.37, "net_price": 0.6489},
                    {"year": 2004, "production_millions": 9.241, "avg_temp_c": 18.86, "avg_precip_mm": 59.6, "avg_producer_price": 2.399, "net_price": 0.5024},
                    {"year": 2005, "production_millions": 9.535, "avg_temp_c": 17.81, "avg_precip_mm": 66.85, "avg_producer_price": 2.399, "net_price": 0.4870},
                    {"year": 2006, "production_millions": 9.74, "avg_temp_c": 18.82, "avg_precip_mm": 127.2, "avg_producer_price": 2.399, "net_price": 0.4701},
                    {"year": 2007, "production_millions": 9.803, "avg_temp_c": 19.10, "avg_precip_mm": 63.2, "avg_producer_price": 2.64, "net_price": 0.6194},
                    {"year": 2008, "production_millions": 10.141, "avg_temp_c": 19.12, "avg_precip_mm": 68.2, "avg_producer_price": 2.525, "net_price": 0.6503},
                    {"year": 2009, "production_millions": 10.42, "avg_temp_c": 19.07, "avg_precip_mm": 76.4, "avg_producer_price": 2.41, "net_price": 0.6357},
                    {"year": 2010, "production_millions": 10.92, "avg_temp_c": 18.96, "avg_precip_mm": 72.9, "avg_producer_price": 2.51, "net_price": 0.6747},
                    {"year": 2011, "production_millions": 11.308, "avg_temp_c": 18.95, "avg_precip_mm": 76.6, "avg_producer_price": 2.44, "net_price": 0.7812},
                    {"year": 2012, "production_millions": 11.591, "avg_temp_c": 18.88, "avg_precip_mm": 75.05, "avg_producer_price": 2.11, "net_price": 0.8200},
                    {"year": 2013, "production_millions": 12.02, "avg_temp_c": 19.03, "avg_precip_mm": 77.5, "avg_producer_price": 2.265, "net_price": 0.9730},
                    {"year": 2014, "production_millions": 12.13, "avg_temp_c": 19.20, "avg_precip_mm": 73.5, "avg_producer_price": 2.19, "net_price": 0.9794},
                    {"year": 2015, "production_millions": 11.896, "avg_temp_c": 19.27, "avg_precip_mm": 75.5, "avg_producer_price": 1.965, "net_price": 0.9495},
                    {"year": 2016, "production_millions": 11.503, "avg_temp_c": 19.20, "avg_precip_mm": 74.1, "avg_producer_price": 2.215, "net_price": 1.2200},
                    {"year": 2017, "production_millions": 11.403, "avg_temp_c": 19.00, "avg_precip_mm": 76.4, "avg_producer_price": 2.18, "net_price": 1.1560},
                    {"year": 2018, "production_millions": 11.447, "avg_temp_c": 18.94, "avg_precip_mm": 77.2, "avg_producer_price": 2.23, "net_price": 1.3096},
                    {"year": 2019, "production_millions": 11.948, "avg_temp_c": 19.10, "avg_precip_mm": 77.6, "avg_producer_price": 2.3, "net_price": 1.4025},
                    {"year": 2020, "production_millions": 12.172, "avg_temp_c": 19.11, "avg_precip_mm": 78.6, "avg_producer_price": 2.545, "net_price": 1.7322},
                    {"year": 2021, "production_millions": 11.962, "avg_temp_c": 19.01, "avg_precip_mm": 79.1, "avg_producer_price": 2.385, "net_price": 2.1530},
                    {"year": 2022, "production_millions": 11.624, "avg_temp_c": 19.07, "avg_precip_mm": 77.0, "avg_producer_price": 2.665, "net_price": 2.6841},
                    {"year": 2023, "production_millions": 11.691, "avg_temp_c": 18.93, "avg_precip_mm": 75.7, "avg_producer_price": 2.53, "net_price": 2.4259}
                ]
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [-51.2177, -28.4284]
            },
            "properties": {
                "region": "Sul",
                "country": "Brazil",
                "timeseries": [
                    {"year": 2000, "production_millions": 4.904, "avg_temp_c": 22.88, "avg_precip_mm": 121.82, "avg_producer_price": 2.286, "net_price": 0.5176},
                    {"year": 2001, "production_millions": 5.188, "avg_temp_c": 23.42, "avg_precip_mm": 156.93, "avg_producer_price": 2.223, "net_price": 0.5340},
                    {"year": 2002, "production_millions": 5.508, "avg_temp_c": 24.55, "avg_precip_mm": 251.9, "avg_producer_price": 2.191, "net_price": 0.5708},
                    {"year": 2003, "production_millions": 5.779, "avg_temp_c": 22.98, "avg_precip_mm": 156.6, "avg_producer_price": 2.191, "net_price": 0.6329},
                    {"year": 2004, "production_millions": 6.246, "avg_temp_c": 23.07, "avg_precip_mm": 115.1, "avg_producer_price": 2.223, "net_price": 0.4913},
                    {"year": 2005, "production_millions": 6.592, "avg_temp_c": 22.33, "avg_precip_mm": 135.3, "avg_producer_price": 2.223, "net_price": 0.4554},
                    {"year": 2006, "production_millions": 7.039, "avg_temp_c": 22.40, "avg_precip_mm": 86.9, "avg_producer_price": 2.223, "net_price": 0.4361},
                    {"year": 2007, "production_millions": 7.51, "avg_temp_c": 22.60, "avg_precip_mm": 115.2, "avg_producer_price": 2.297, "net_price": 0.5852},
                    {"year": 2008, "production_millions": 8.268, "avg_temp_c": 22.09, "avg_precip_mm": 110.8, "avg_producer_price": 2.227, "net_price": 0.6201},
                    {"year": 2009, "production_millions": 8.957, "avg_temp_c": 22.20, "avg_precip_mm": 125.75, "avg_producer_price": 2.273, "net_price": 0.6247},
                    {"year": 2010, "production_millions": 9.611, "avg_temp_c": 22.17, "avg_precip_mm": 133.4, "avg_producer_price": 2.347, "net_price": 0.6623},
                    {"year": 2011, "production_millions": 10.226, "avg_temp_c": 22.06, "avg_precip_mm": 134.4, "avg_producer_price": 2.29, "net_price": 0.7660},
                    {"year": 2012, "production_millions": 10.736, "avg_temp_c": 22.28, "avg_precip_mm": 125.7, "avg_producer_price": 1.98, "net_price": 0.7985},
                    {"year": 2013, "production_millions": 11.774, "avg_temp_c": 22.16, "avg_precip_mm": 128.45, "avg_producer_price": 2.13, "net_price": 0.9447},
                    {"year": 2014, "production_millions": 12.211, "avg_temp_c": 22.18, "avg_precip_mm": 129.2, "avg_producer_price": 2.06, "net_price": 0.9584},
                    {"year": 2015, "production_millions": 12.319, "avg_temp_c": 22.42, "avg_precip_mm": 138.85, "avg_producer_price": 1.817, "net_price": 0.9290},
                    {"year": 2016, "production_millions": 12.454, "avg_temp_c": 22.48, "avg_precip_mm": 98.0, "avg_producer_price": 2.127, "net_price": 1.2208},
                    {"year": 2017, "production_millions": 11.78, "avg_temp_c": 22.47, "avg_precip_mm": 128.85, "avg_producer_price": 2.057, "net_price": 1.1423},
                    {"year": 2018, "production_millions": 11.598, "avg_temp_c": 22.33, "avg_precip_mm": 137.2, "avg_producer_price": 2.09, "net_price": 1.2898},
                    {"year": 2019, "production_millions": 11.692, "avg_temp_c": 22.52, "avg_precip_mm": 129.8, "avg_producer_price": 2.23, "net_price": 1.3981},
                    {"year": 2020, "production_millions": 12.058, "avg_temp_c": 22.52, "avg_precip_mm": 130.2, "avg_producer_price": 2.513, "net_price": 1.7367},
                    {"year": 2021, "production_millions": 11.978, "avg_temp_c": 22.51, "avg_precip_mm": 116.1, "avg_producer_price": 2.28, "net_price": 2.1248},
                    {"year": 2022, "production_millions": 11.666, "avg_temp_c": 22.43, "avg_precip_mm": 140.75, "avg_producer_price": 2.547, "net_price": 2.6595},
                    {"year": 2023, "production_millions": 11.878, "avg_temp_c": 22.50, "avg_precip_mm": 88.6, "avg_producer_price": 2.417, "net_price": 2.4134}
                ]
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [-51.9253, -3.4653]
            },
            "properties": {
                "region": "Norte",
                "country": "Brazil",
                "timeseries": [
                    {"year": 2000, "production_millions": 1.05, "avg_temp_c": 26.47, "avg_precip_mm": 137.8, "net_price": 0.0000},
                    {"year": 2001, "production_millions": 1.237, "avg_temp_c": 26.98, "avg_precip_mm": 147.0, "net_price": 0.0000},
                    {"year": 2002, "production_millions": 1.567, "avg_temp_c": 26.91, "avg_precip_mm": 147.0, "net_price": 0.0000},
                    {"year": 2003, "production_millions": 1.498, "avg_temp_c": 26.37, "avg_precip_mm": 164.9,"net_price": 0.0000 },
                    {"year": 2004, "production_millions": 1.663, "avg_temp_c": 26.23, "avg_precip_mm": 164.9, "net_price": 0.0000},
                    {"year": 2005, "production_millions": 1.743, "avg_temp_c": 26.43, "avg_precip_mm": 131.0,"net_price": 0.0000 },
                    {"year": 2006, "production_millions": 1.699, "avg_temp_c": 26.47, "avg_precip_mm": 131.6,"net_price": 0.0000 },
                    {"year": 2007, "production_millions": 1.677, "avg_temp_c": 26.32, "avg_precip_mm": 81.2, "net_price": 0.0000 },
                    {"year": 2008, "production_millions": 1.666, "avg_temp_c": 26.41, "avg_precip_mm": 103.75,"net_price": 0.0000 },
                    {"year": 2009, "production_millions": 1.673, "avg_temp_c": 26.36, "avg_precip_mm": 141.0, "net_price": 0.0000},
                    {"year": 2010, "production_millions": 1.737, "avg_temp_c": 26.37, "avg_precip_mm": 128.25,"net_price": 0.0000 },
                    {"year": 2011, "production_millions": 1.675, "avg_temp_c": 26.32, "avg_precip_mm": 134.6,"net_price": 0.0000 },
                    {"year": 2012, "production_millions": 1.658, "avg_temp_c": 26.31, "avg_precip_mm": 133.15,"net_price": 0.0000 },
                    {"year": 2013, "production_millions": 1.846, "avg_temp_c": 26.32, "avg_precip_mm": 134.25,"net_price": 0.0000},
                    {"year": 2014, "production_millions": 1.946, "avg_temp_c": 26.30, "avg_precip_mm": 140.3,"net_price": 0.0000 },
                    {"year": 2015, "production_millions": 1.833, "avg_temp_c": 26.36, "avg_precip_mm": 126.7,"net_price": 0.0000},
                    {"year": 2016, "production_millions": 1.876, "avg_temp_c": 26.38, "avg_precip_mm": 125.6, "net_price": 0.0000},
                    {"year": 2017, "production_millions": 2.182, "avg_temp_c": 26.37, "avg_precip_mm": 121.0, "net_price": 0.0000},
                    {"year": 2018, "production_millions": 2.294, "avg_temp_c": 26.34, "avg_precip_mm": 124.1, "net_price": 0.0000},
                    {"year": 2019, "production_millions": 2.239, "avg_temp_c": 26.35, "avg_precip_mm": 127.3, "net_price": 0.0000},
                    {"year": 2020, "production_millions": 2.055, "avg_temp_c": 26.35, "avg_precip_mm": 132.0, "net_price": 0.0000},
                    {"year": 2021, "production_millions": 1.842, "avg_temp_c": 26.33, "avg_precip_mm": 135.8, "net_price": 0.0000 },
                    {"year": 2022, "production_millions": 1.757, "avg_temp_c": 26.33, "avg_precip_mm": 135.8,"net_price": 0.0000 },
                    {"year": 2023, "production_millions": 1.745, "avg_temp_c": 26.34, "avg_precip_mm": 130.2, "net_price": 0.0000}
                ]
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [-41.9, -9.4]
            },
            "properties": {
                "region": "Nordeste",
                "country": "Brazil",
                "timeseries": [
                   {"year": 2000, "production_millions": 2.159, "avg_temp_c": 25.8, "avg_precip_mm": 74.28, "net_price": 0.4766, "avg_producer_price": 2.19},
                   {"year": 2001, "production_millions": 2.266, "avg_temp_c": 25.43, "avg_precip_mm": 167.3, "net_price": 0.4968, "avg_producer_price": 2.15},
                   {"year": 2002, "production_millions": 2.363, "avg_temp_c": 25.07, "avg_precip_mm": 22.8, "net_price": 0.5395, "avg_producer_price": 2.136},
                   {"year": 2003, "production_millions": 2.508, "avg_temp_c": 25.47, "avg_precip_mm": 55.7, "net_price": 0.5876, "avg_producer_price": 2.136},
                   {"year": 2004, "production_millions": 2.705, "avg_temp_c": 25.75, "avg_precip_mm": 63.8, "net_price": 0.4481, "avg_producer_price": 2.15},
                   {"year": 2005, "production_millions": 2.972, "avg_temp_c": 25.98, "avg_precip_mm": 63.6, "net_price": 0.4272, "avg_producer_price": 2.15},
                   {"year": 2006, "production_millions": 3.198, "avg_temp_c": 26.04, "avg_precip_mm": 57.3, "net_price": 0.3868, "avg_producer_price": 2.15},
                   {"year": 2007, "production_millions": 3.339, "avg_temp_c": 26.38, "avg_precip_mm": 21.0, "net_price": 0.5211, "avg_producer_price": 2.17},
                   {"year": 2008, "production_millions": 3.455, "avg_temp_c": 25.83, "avg_precip_mm": 25.9, "net_price": 0.6, "avg_producer_price": 2.27},
                   {"year": 2009, "production_millions": 3.813, "avg_temp_c": 26.0, "avg_precip_mm": 41.9, "net_price": 0.5966, "avg_producer_price": 2.21},
                   {"year": 2010, "production_millions": 3.998, "avg_temp_c": 26.17, "avg_precip_mm": 45.2, "net_price": 0.6256, "avg_producer_price": 2.26},
                   {"year": 2011, "production_millions": 4.11, "avg_temp_c": 26.03, "avg_precip_mm": 49.2, "net_price": 0.6678, "avg_producer_price": 2.04},
                   {"year": 2012, "production_millions": 3.501, "avg_temp_c": 26.12, "avg_precip_mm": 37.9, "net_price": 0.7983, "avg_producer_price": 1.97},
                   {"year": 2013, "production_millions": 3.598, "avg_temp_c": 26.14, "avg_precip_mm": 41.8, "net_price": 0.9294, "avg_producer_price": 2.16},
                   {"year": 2014, "production_millions": 3.892, "avg_temp_c": 26.13, "avg_precip_mm": 42.0, "net_price": 0.9776, "avg_producer_price": 2.23},
                   {"year": 2015, "production_millions": 3.957, "avg_temp_c": 26.28, "avg_precip_mm": 38.6, "net_price": 0.9349, "avg_producer_price": 1.9},
                   {"year": 2016, "production_millions": 3.875, "avg_temp_c": 26.34, "avg_precip_mm": 34.5, "net_price": 1.1153, "avg_producer_price": 1.94},
                   {"year": 2017, "production_millions": 3.981, "avg_temp_c": 26.36, "avg_precip_mm": 34.4, "net_price": 1.1535, "avg_producer_price": 2.04},
                   {"year": 2018, "production_millions": 4.476, "avg_temp_c": 26.3, "avg_precip_mm": 34.6, "net_price": 1.2499, "avg_producer_price": 2.01},
                   {"year": 2019, "production_millions": 4.852, "avg_temp_c": 26.34, "avg_precip_mm": 35.55, "net_price": 1.338, "avg_producer_price": 2.13},
                   {"year": 2020, "production_millions": 4.919, "avg_temp_c": 26.34, "avg_precip_mm": 36.6, "net_price": 1.2826, "avg_producer_price": 2.51},
                   {"year": 2021, "production_millions": 5.419, "avg_temp_c": 26.33, "avg_precip_mm": 37.0, "net_price": 0.9917, "avg_producer_price": 2.17},
                   {"year": 2022, "production_millions": 5.697, "avg_temp_c": 26.35, "avg_precip_mm": 36.6, "net_price": 2.4106, "avg_producer_price": 2.34},
                   {"year": 2023, "production_millions": 6.287, "avg_temp_c": 26.36, "avg_precip_mm": 35.0, "net_price": 2.2376, "avg_producer_price": 2.26}
                ]
            }
        }
    ]
}

url = "https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=UF"

data = requests.get(url).json()

print(data)

m = folium.Map(location=[-15, -48], zoom_start=4, max_zoom=4)

# folium.GeoJson AUTOMATICALLY fixes the coordinates
folium.GeoJson(data).add_to(m)

colors = {
    'Centro-Oeste': 'orange',
    'Sudeste': 'blue',
    'Sul': 'green',
    'Norte': 'red',
    'Nordeste': 'purple'
}

def get_region_from_code(state_code):
    # Ensure code is a string and take the first character
    first_digit = str(state_code)[0]

    mapping = {
        '1': 'Norte',
        '2': 'Nordeste',
        '3': 'Sudeste',
        '4': 'Sul',
        '5': 'Centro-Oeste'
    }
    return mapping.get(first_digit)

# 4. Create the Style Function
# This runs for every single state (feature) in the GeoJSON
def my_style_function(feature):
    # Get the code (e.g., "35" for SP)
    cod_area = feature['properties']['codarea']

    # Find out which region it belongs to
    region_name = get_region_from_code(cod_area)

    # Get the color from your dictionary (default to gray if error)
    fill_color = colors.get(region_name, 'gray')

    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# 5. Generate Map
m = folium.Map(location=[-15, -48], zoom_start=4)

folium.GeoJson(
    data,
    style_function=my_style_function,
    tooltip=folium.GeoJsonTooltip(fields=['codarea'], aliases=['State Code:'])
).add_to(m)

for feature in geojson_data['features']:
    coords = feature['geometry']['coordinates']
    props = feature['properties']
    region = props['region']
    timeseries = props['timeseries']

    popup_html = f"""
    <div style="font-family: Arial; width: 400px;  max-height: 500px; overflow-y: auto;">
        <h3 style="margin: 0; color: {colors[region]}; position: sticky; top: 0; background: white; padding: 10px 0;">{region}</h3>
        <hr style="margin: 5px 0;">
        <table style="width: 100%; border-collapse: collapse; font-size: 11px;">
            <thead style="position: sticky; top: 45px; background: white;">
                <tr style="background-color: #f0f0f0;">
                    <th style="border: 1px solid #ddd; padding: 5px;">Ano</th>
                    <th style="border: 1px solid #ddd; padding: 5px;">Produção<br>(M. litros)</th>
                    <th style="border: 1px solid #ddd; padding: 5px;">Temp.<br>(°C)</th>
                    <th style="border: 1px solid #ddd; padding: 5px;">Precip.<br>(mm)</th>
    """

    if 'avg_producer_price' in timeseries[0]:
        popup_html += """
                    <th style="border: 1px solid #ddd; padding: 5px;">Custo de produção</th>
                    <th style="border: 1px solid #ddd; padding: 5px;">Valor comercializado entre produtores<br>(R$ por litro) </th>
        """

    popup_html += """
                </tr>
            </thead>
            <tbody>
    """

    for data in timeseries:
        row_color = "#f9f9f9" if data['year'] % 2 == 0 else "white"
        popup_html += f"""
                <tr style="background-color: {row_color};">
                    <td style="border: 1px solid #ddd; padding: 5px; text-align: center;"><b>{int(data['year'])}</b></td>
                    <td style="border: 1px solid #ddd; padding: 5px; text-align: right;">{data['production_millions']:.3f}</td>
                    <td style="border: 1px solid #ddd; padding: 5px; text-align: right;">{data['avg_temp_c']:.2f}</td>
                    <td style="border: 1px solid #ddd; padding: 5px; text-align: right;">{data['avg_precip_mm']:.2f}</td>
        """

        if 'avg_producer_price' in data:
            popup_html += f"""
                    <td style="border: 1px solid #ddd; padding: 5px; text-align: right;">{data['avg_producer_price']:.3f}</td>
                    <td style="border: 1px solid #ddd; padding: 5px; text-align: right;">{data['net_price']:.4f}</td>
            """

        popup_html += "</tr>"

    popup_html += """
            </tbody>
        </table>
        <div style="margin-top: 10px; font-size: 11px; color: #666;">
            <b>Sumário:</b> Desde 2000 Aumentou de {:.2f}M para {:.2f}M litros em 2023 ({:+.1f}%)
        </div>
    </div>
    """.format(
        timeseries[0]['production_millions'],
        timeseries[-1]['production_millions'],
        ((timeseries[-1]['production_millions'] - timeseries[0]['production_millions']) / timeseries[0]['production_millions'] * 100)
    )

    last_year = timeseries[-1]

    folium.CircleMarker(
        location=[coords[1], coords[0]],
        radius=45 + (last_year['production_millions'] / 1.1),
        popup=folium.Popup(popup_html, max_width=650),
        tooltip=f"Produção total do {region}: {last_year['production_millions']:.2f}M litros em (2023)",
        color=colors[region],
        fill=True,
        fillColor=colors[region],
        fillOpacity=0.0,
        weight=0
    ).add_to(m)

    folium.Marker(
        location=[coords[1], coords[0]],
        popup=folium.Popup(popup_html, max_width=650),
        icon=folium.DivIcon(html=f"""
            <div style="font-size: 13pt;
                        color: white;
                        font-weight: bold;
                        text-shadow: 0 0 3px white;
                        z-index: 9999;
                        cursor: pointer;">
                {region}
            </div>
        """)
    ).add_to(m)

title_html = '''
<div style="position: fixed;
            top: 10px; left: 50px; width: 500px; height: 120px;
            background-color: white; border:2px solid grey; z-index:9999;
            font-size:14px; padding: 10px">
<h3 style="margin: 0;">Produção de leite por região</h3>
<p style="margin: 5px 0;">Os dados vão de 2000 até 2023<br>
<b>Clique nos nomes das regiões para visualizar seus respectivos dados</b></p>
</div>
'''


#Aqui colocamos os marcadores de embrapa no mapa
icon = folium.CustomIcon(
    icon_image="/content/embrapa.png",
    icon_size=(30, 22),

)

m.get_root().html.add_child(folium.Element(title_html))

for cidade in set(lista_cidades):
    if cidade in coordenadas_cidades:
        lat, lon = coordenadas_cidades[cidade]
        folium.Marker(
            location=[lat, lon],
            popup=f"Unidade da Embrapa em: {cidade}",
            icon=icon,
            tooltip=cidade
        ).add_to(m)

icon2 = folium.CustomIcon(
    icon_image="/content/logo-historia.png",
        icon_size=(25, 25),
)

for cidade, coordenadas in locais.items():
    folium.Marker(
        location=coordenadas,
        popup=f"Unidade da RBQL em: {cidade}",            # Texto ao clicar
        tooltip=cidade,          # Texto ao passar o mouse
        icon=icon2,
    ).add_to(m)



# Save the map
m.save('milk_production_map.html')
print("Map saved as 'milk_production_map.html'")
print("Open this file in your web browser to view the interactive map!")

# Also print summary statistics
print("\n=== Summary Statistics (2023) ===")
for feature in geojson_data['features']:
    props = feature['properties']
    last_year = props['timeseries'][-1]
    print(f"{props['region']:15s}: {last_year['production_millions']:6.2f} million liters")

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-62.4177, -13.1189], [-62.2785, -13.1555], [-62.2148, -13.1113], [-62.1152, -13.1637], [-62.1157, -13.2585], [-61.8874, -13.4408], [-61.8171, -13.5274], [-61.5789, -13.5099], [-61.4699, -13.5553], [-61.2483, -13.4963], [-61.1943, -13.5364], [-61.009, -13.5064], [-60.919, -13.5483], [-60.8798, -13.6175], [-60.7093, -13.693], [-60.6322, -13.5717], [-60.3879, -13.4547], [-60.3716, -13.3186], [-60.3332, -13.258], [-60.2676, -13.0773], [-60.1833, -12.9676], [-60.1172, -12.9596], [-60.0289, -12.6186], [-59.9477, -12.5993], [-59.7794, -12.3415], [-59.8868, -12.2451], [-59.8998, -12.1172], [-59.9794, -12.0295], [-59.9842, -11.9148], [-60.0989, -11.8456], [-60.1109, -11.5825], [-59.9344, -11.4237], [-59.9172, -11.3384], [-59.9797, -11.2395], [-59.9768, -11.1224], [-60.3916, -11.0938], [-60.4601, -10.9899], [-61.0006, -10.9922], [-61.429, -10.9875], [-61.5503, -10.9861], [-61.4766, -1

# Parte de execução dos algoritmos de IA

In [ ]:
dataframe_leite.to_csv("dataframe_leite.csv", index=False)

In [ ]:
dataframe_leite['Preco_SUD_Lag1'] = dataframe_leite['média do preço do leite para o produtor no Sudeste'].shift(1)
dataframe_leite['Producao_SUD_Lag1'] = dataframe_leite['média da produção de leite sudeste (milhões)'].shift(1)

dataframe_leite

In [ ]:
variable = "média do preço do leite para o produtor no Sudeste"
y = dataframe_leite[variable]

features = ["média do preço do leite para o produtor no Sudeste", "Temperatura média SUDESTE (°C)", "ipca_variacao", "Preco_SUD_Lag1", "Producao_SUD_Lag1"]

In [ ]:
x  = dataframe_leite[features]
x

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

## 6. Make Predictions and Evaluate
y_pred = rf_model.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

Root Mean Squared Error (RMSE): 0.0299


In [ ]:
print("\n--- Example Predictions ---")
print(f"Actual values (y_test):    {y_test.values}")
print(f"Predicted values (y_pred): {[round(p, 2) for p in y_pred]}")


--- Example Predictions ---
Actual values (y_test):    [2.525 2.215 2.505 2.23  2.44  2.41  2.265 2.399]
Predicted values (y_pred): [np.float64(2.53), np.float64(2.17), np.float64(2.52), np.float64(2.18), np.float64(2.41), np.float64(2.4), np.float64(2.23), np.float64(2.38)]


In [ ]:
print("\n--- Feature Importance ---")
# Get and display the importance scores
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)


--- Feature Importance ---
                                             Feature  Importance
0  média do preço do leite para o produtor no Sud...    0.840595
3                                     Preco_SUD_Lag1    0.066194
4                                  Producao_SUD_Lag1    0.060306
1                     Temperatura média SUDESTE (°C)    0.021298
2                                      ipca_variacao    0.011607
